In [8]:
import os
import json
import csv
import pandas as pd

# Rough ground for testing things out

In [9]:
test_file = './mlm_analysis/adv_dec/region_results.json'

In [10]:
with open(test_file, 'r') as f:
    data = json.load(f)

In [11]:
results = {}
scores = {}

In [12]:
for identity_term in data:
    results[identity_term] = {}
    scores[identity_term] = {}
    for subcategory in data[identity_term]:
        results[identity_term][subcategory] = {}
        scores[identity_term][subcategory] = {}
        for datum in data[identity_term][subcategory]:
            for top_results in datum:
                token = top_results['token_str']
                score = top_results['score']
                if token not in results[identity_term][subcategory]:
                    results[identity_term][subcategory][token] = 0
                    scores[identity_term][subcategory][token] = 0

                results[identity_term][subcategory][token] += 1
                scores[identity_term][subcategory][token] += score

In [13]:
# sort results by highest occurence
for identity_term in results:
    for subcategory in results[identity_term]:
        results[identity_term][subcategory] = {k: v for k, v in sorted(results[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}
        scores[identity_term][subcategory] = {k: v for k, v in sorted(scores[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}

# Compiling the results for each of the experiments

In [18]:
subcategory_map = {
    'professions': 'professions',
    'subjects_of_study': 'subjects',
    'action_verbs': 'verbs',
    'behaviour_adjectives': 'adjectives',
    'socio_economic_status_adjectives': 'socio_economic_status',
    'food_habits_adjectives': 'food',
    'clothing_preferences_adjectives': 'clothes',
    'general': 'general'
}

In [20]:
for folder in os.listdir('./mlm_analysis'):
    print(folder)
    os.makedirs(f'./mlm_analysis/{folder}/results', exist_ok=True)

    if 'combined' not in folder:
        files_list = ['region_results.json', 'religion_results.json']
        for file in files_list:
            file_path = os.path.join('./mlm_analysis', folder, file)
            with open(file_path, 'r') as f:
                data = json.load(f)

            results = {}
            scores = {}

            for identity_term in data:
                results[identity_term] = {}
                scores[identity_term] = {}
                for subcategory in data[identity_term]:
                    subcategory_term = subcategory_map[subcategory]

                    results[identity_term][subcategory_term] = {}
                    scores[identity_term][subcategory_term] = {}
                    for datum in data[identity_term][subcategory]:
                        for top_results in datum:
                            try:
                                token = top_results['token_str'].lower()
                                score = top_results['score']
                            except:
                                print(datum)

                            if token not in results[identity_term][subcategory_term]:
                                results[identity_term][subcategory_term][token] = 0
                                scores[identity_term][subcategory_term][token] = 0

                            results[identity_term][subcategory_term][token] += 1
                            scores[identity_term][subcategory_term][token] += score

            # sort results by highest occurence
            for identity_term in results:
                for subcategory in results[identity_term]:
                    results[identity_term][subcategory] = {k: v for k, v in sorted(results[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}
                    scores[identity_term][subcategory] = {k: v for k, v in sorted(scores[identity_term][subcategory].items(), key=lambda item: item[1], reverse=True)}

            with open(f'./mlm_analysis/{folder}/results/{file}', 'w') as f:
                json.dump(results, f, indent=4)
            with open(f'./mlm_analysis/{folder}/results/scores_{file}', 'w') as f:
                json.dump(scores, f, indent=4)
    else:
        files_list = 'combined_results.json'
        file_path = os.path.join('./mlm_analysis', folder, files_list)
        with open(file_path, 'r') as f:
            data = json.load(f)

        results = {}
        scores = {}

        for identity_term_1 in data:
            results[identity_term_1] = {}
            scores[identity_term_1] = {}
            for identity_term_2 in data[identity_term_1]:
                results[identity_term_1][identity_term_2] = {}
                scores[identity_term_1][identity_term_2] = {}
                for subcategory in data[identity_term_1][identity_term_2]:
                    subcategory_term = subcategory_map[subcategory]
                    results[identity_term_1][identity_term_2][subcategory_term] = {}
                    scores[identity_term_1][identity_term_2][subcategory_term] = {}
                    for datum in data[identity_term_1][identity_term_2][subcategory]:
                        for top_results in datum:
                            token = top_results['token_str']
                            score = top_results['score']
                            if token not in results[identity_term_1][identity_term_2][subcategory_term]:
                                results[identity_term_1][identity_term_2][subcategory_term][token] = 0
                                scores[identity_term_1][identity_term_2][subcategory_term][token] = 0

                            results[identity_term_1][identity_term_2][subcategory_term][token] += 1
                            scores[identity_term_1][identity_term_2][subcategory_term][token] += score

        # sort results by highest occurence
        for identity_term_1 in results:
            for identity_term_2 in results[identity_term_1]:
                for subcategory in results[identity_term_1][identity_term_2]:
                    results[identity_term_1][identity_term_2][subcategory] = {k: v for k, v in sorted(results[identity_term_1][identity_term_2][subcategory].items(), key=lambda item: item[1], reverse=True)}
                    scores[identity_term_1][identity_term_2][subcategory] = {k: v for k, v in sorted(scores[identity_term_1][identity_term_2][subcategory].items(), key=lambda item: item[1], reverse=True)}

        with open(f'./mlm_analysis/{folder}/results/{files_list}', 'w') as f:
            json.dump(results, f, indent=4)
        with open(f'./mlm_analysis/{folder}/results/scores_{files_list}', 'w') as f:
            json.dump(scores, f, indent=4)

negative_framing
positive_framing
adv_inc
adv_perspective_shift
adv_dec
adv_future
vanilla
combined
adv_present
neutral_framing
adv_past


# Finding stereotype scores for the original dataset

In [3]:
region_token_annotations = {}

with open('./mlm_analysis/region_annotated.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'category':
            continue
        region_token_annotations[row[1]] = row[0]

len(region_token_annotations)

139

In [4]:
religion_token_annotations = {}

with open('./mlm_analysis/religion_annotated.csv', 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        if row[0] == 'category':
            continue
        religion_token_annotations[row[1]] = row[0]

len(religion_token_annotations)

216

In [6]:
all_region_data = {}

with open('./nlp-fairness-for-india/region_annotations.tsv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] == 'identity_term':
            continue

        identity_term = row[0]
        identity_term = identity_term[0].upper() + identity_term[1:]

        token = row[1]
        stereotype_votes = int(row[2])
        non_stereotype_votes = int(row[3])
        total_votes = int(row[5])

        if stereotype_votes > non_stereotype_votes:
            stereotype = True
            annotation_confidence = stereotype_votes / total_votes if total_votes > 0 else 0
        else:
            stereotype = False
            annotation_confidence = non_stereotype_votes / total_votes if total_votes > 0 else 0
        
        if identity_term not in all_region_data:
            all_region_data[identity_term] = []

        all_region_data[identity_term].append({
            'token': token,
            'stereotype': stereotype,
            'annotation_confidence': annotation_confidence,
            'annotation': region_token_annotations.get(token, None),
        })    

json.dump(all_region_data, open('./mlm_analysis/all_region_data.json', 'w'))

In [7]:
all_religion_data = {}

with open('./nlp-fairness-for-india/religion_annotations.tsv', 'r') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        if row[0] == 'identity_term':
            continue

        identity_term = row[0]
        identity_term = identity_term[0].upper() + identity_term[1:]

        token = row[1]
        stereotype_votes = int(row[2])
        non_stereotype_votes = int(row[3])
        total_votes = int(row[5])

        if stereotype_votes > non_stereotype_votes:
            stereotype = True
            annotation_confidence = stereotype_votes / total_votes if total_votes > 0 else 0
        else:
            stereotype = False
            annotation_confidence = non_stereotype_votes / total_votes if total_votes > 0 else 0
        
        if identity_term not in all_religion_data:
            all_religion_data[identity_term] = []

        all_religion_data[identity_term].append({
            'token': token,
            'stereotype': stereotype,
            'annotation_confidence': annotation_confidence,
            'annotation': religion_token_annotations.get(token, None),
        })

json.dump(all_religion_data, open('./mlm_analysis/all_religion_data.json', 'w'))

# Calculating scores for the experiments - Rough ground for testing things out

In [27]:
test_file = './mlm_analysis/adv_future/results/region_results.json'
test_data = json.load(open(test_file, 'r'))

In [28]:
stereotype_scores = {}
stereotype_tokens = {}

In [29]:
for identity_term in all_region_data:
    stereotype_scores[identity_term] = {}
    stereotype_tokens[identity_term] = {}
    for token_data in all_region_data[identity_term]:
        token = token_data['token']
        annotation = token_data['annotation']
        stereotype = token_data['stereotype']
        annotation_confidence = token_data['annotation_confidence']
        if stereotype and token in test_data[identity_term][annotation]:
            if annotation not in stereotype_scores[identity_term]:
                stereotype_scores[identity_term][annotation] = 0
                stereotype_tokens[identity_term][annotation] = []
            stereotype_scores[identity_term][annotation] += test_data[identity_term][annotation][token] * annotation_confidence
            stereotype_tokens[identity_term][annotation].append(token)

for identity_term in stereotype_scores:
    stereotype_scores[identity_term] = {k: v for k, v in sorted(stereotype_scores[identity_term].items(), key=lambda item: item[1], reverse=True)}

In [30]:
stereotype_scores

{'Arunachali': {},
 'Assamese': {},
 'Bengali': {'subjects': 0.3333333333333333},
 'Bihari': {'adjectives': 0.3333333333333333},
 'Chattisgarhi': {},
 'Goan': {'subjects': 0.6666666666666666},
 'Gujarati': {},
 'Haryanvi': {'subjects': 0.3333333333333333},
 'Himachali': {},
 'Jharkhandi': {},
 'Kannadiga': {'subjects': 1.0},
 'Kashmiri': {},
 'Madhya pradeshi': {},
 'Manipuri': {'subjects': 0.3333333333333333},
 'Marathi': {},
 'Meghalayan': {},
 'Mizo': {},
 'Odiya': {},
 'Punjabi': {'subjects': 0.3333333333333333},
 'Rajasthani': {'subjects': 0.6666666666666666,
  'adjectives': 0.3333333333333333},
 'Tamilian': {'subjects': 1.6666666666666665},
 'Telugu': {},
 'Tripuri': {},
 'Uttar pradeshi': {'professions': 0.6666666666666666},
 'Uttarakhandi': {}}

In [31]:
stereotype_tokens

{'Arunachali': {},
 'Assamese': {},
 'Bengali': {'subjects': ['art']},
 'Bihari': {'adjectives': ['poor']},
 'Chattisgarhi': {},
 'Goan': {'subjects': ['english']},
 'Gujarati': {},
 'Haryanvi': {'subjects': ['music']},
 'Himachali': {},
 'Jharkhandi': {},
 'Kannadiga': {'subjects': ['english']},
 'Kashmiri': {},
 'Madhya pradeshi': {},
 'Manipuri': {'subjects': ['art']},
 'Marathi': {},
 'Meghalayan': {},
 'Mizo': {},
 'Odiya': {},
 'Punjabi': {'subjects': ['music']},
 'Rajasthani': {'subjects': ['history', 'music'], 'adjectives': ['rich']},
 'Tamilian': {'subjects': ['english', 'science']},
 'Telugu': {},
 'Tripuri': {},
 'Uttar pradeshi': {'professions': ['farmer']},
 'Uttarakhandi': {}}

# Calculating these scores for all the experiments

In [38]:
for folder in os.listdir('./mlm_analysis'):
    print(folder)
    if os.path.isdir(f'./mlm_analysis/{folder}'):
        os.makedirs(f'./mlm_analysis/{folder}/final_results', exist_ok=True)

        if 'combined' not in folder:
            files_list = ['region_results.json']
            for file in files_list:
                file_path = os.path.join('./mlm_analysis', folder, 'results', file)

                results_data = json.load(open(file_path, 'r'))

                stereotype_scores = {}
                stereotype_tokens = {}

                for identity_term in all_region_data:
                    stereotype_scores[identity_term] = {}
                    stereotype_tokens[identity_term] = {}
                    for token_data in all_region_data[identity_term]:
                        token = token_data['token']
                        annotation = token_data['annotation']
                        stereotype = token_data['stereotype']
                        annotation_confidence = token_data['annotation_confidence']
                        try:
                            if stereotype and token in results_data[identity_term][annotation]:
                                if annotation not in stereotype_scores[identity_term]:
                                    stereotype_scores[identity_term][annotation] = 0
                                    stereotype_tokens[identity_term][annotation] = []
                                stereotype_scores[identity_term][annotation] += results_data[identity_term][annotation][token] * annotation_confidence
                                stereotype_tokens[identity_term][annotation].append(token)     
                        except Exception as e:
                            print(folder, identity_term, annotation, token)

                with open(f'./mlm_analysis/{folder}/final_results/{file}', 'w') as f:
                    json.dump(stereotype_scores, f, indent=4)
                with open(f'./mlm_analysis/{folder}/final_results/stereotype_tokens_{file}', 'w') as f:
                    json.dump(stereotype_tokens, f, indent=4)   

            files_list = ['religion_results.json']
            for file in files_list:
                file_path = os.path.join('./mlm_analysis', folder, 'results', file)

                results_data = json.load(open(file_path, 'r'))

                stereotype_scores = {}
                stereotype_tokens = {}

                for identity_term in all_religion_data:
                    stereotype_scores[identity_term] = {}
                    stereotype_tokens[identity_term] = {}
                    for token_data in all_religion_data[identity_term]:
                        token = token_data['token']
                        annotation = token_data['annotation']
                        stereotype = token_data['stereotype']
                        annotation_confidence = token_data['annotation_confidence']
                        try:
                            if stereotype and token in results_data[identity_term][annotation]:
                                if annotation not in stereotype_scores[identity_term]:
                                    stereotype_scores[identity_term][annotation] = 0
                                    stereotype_tokens[identity_term][annotation] = []
                                stereotype_scores[identity_term][annotation] += results_data[identity_term][annotation][token] * annotation_confidence
                                stereotype_tokens[identity_term][annotation].append(token)     
                        except Exception as e:
                            print(folder, identity_term, annotation, token)

                with open(f'./mlm_analysis/{folder}/final_results/{file}', 'w') as f:
                    json.dump(stereotype_scores, f, indent=4)
                with open(f'./mlm_analysis/{folder}/final_results/stereotype_tokens_{file}', 'w') as f:
                    json.dump(stereotype_tokens, f, indent=4)


negative_framing
negative_framing Muslim offensive_terms terrorist
positive_framing
positive_framing Muslim offensive_terms terrorist
adv_inc
adv_inc Muslim offensive_terms terrorist
all_region_data.json
adv_perspective_shift
adv_perspective_shift Muslim offensive_terms terrorist
adv_dec
adv_dec Muslim offensive_terms terrorist
region_annotated.csv
adv_future
adv_future Muslim offensive_terms terrorist
vanilla
vanilla Muslim offensive_terms terrorist
combined
religion_annotated.csv
adv_present
adv_present Muslim offensive_terms terrorist
neutral_framing
neutral_framing Muslim offensive_terms terrorist
all_religion_data.json
adv_past
adv_past Muslim offensive_terms terrorist
